### 优先队列数据类型
队列是一种FIFO的数据类型,但这种FIFO的原则在有些情况下并不适用,有时候我们需要根据其他因素觉得出栈顺序.  
一种解决方法是"first come ,first serve"先进先服务  
优先队列就是这种改良的体现:随机入队,优先元素出队--当一个元素入队后,给予其一个权限key,key子最小的元素是特权元素,将会出队.

#### 优先队列的数据模型
* p.add(k,v): 入队新元素
* p.min():    返回key最小元素
* p.remove_min():最小元素出列
* p.is_empty():判断队列是否为空
* len(p):      返回元素个数

#### 优先队列的实现 原型

In [1]:
class PriorityQueueBase:
    class _ltem:
        __slot__="_key","_value"
        
        def __init__(self,k,v):
            self._key=k
            self._value=v
        #以key为基础比较
        def __lt__(self,other):
            return self._key<other._key
        
    def is_empty(self):
        return len(self)==0
    

#### 使用无序list实现
用无序list存储数据,从下图可以看到,方便入列,但出列需要排序
![Screenshot from 2017-03-28 18-19-26.png](https://ooo.0o0.ooo/2017/03/28/58da38be757e3.png)

In [2]:
class UnsortedPriorityQueue(PriorityQueueBase):
    def __init__(self):
        self.data=[]
        
    def add(self,key,value):
        newitem=self._ltem(key,value)
        self.data.append(newitem)
        
    def __len__(self):
        return len(self.data)
    
    def find_min(self):
        if self.is_empty():
            raise Empty("no item")
        minitem=self.data[0]
        for item in self.data:
            #运算符重载
            if item<minitem:
                minitem=item
        return minitem
    
    def min(self):
        minitem= self.find_min()
        return minitem._key,minitem._value
    
    
    def remove_min(self):
        minitem=self.find_min()
        self.data.remove(minitem)
        return minitem._key,minitem._value
    
    def __iter__(self):
        for item in self.data:
            yield item._key,item._value
    
    def __str__(self):
        return " ".join([str(pair) for pair in iter(self)])
    
pq=UnsortedPriorityQueue()
pq.add(1,"a")
print(pq)
pq.add(3,"b")
print(pq)
print(pq.min()) 
pq.remove_min()
print(pq)

(1, 'a')
(1, 'a') (3, 'b')
(1, 'a')
(3, 'b')


#### 使用有序list实现优先队列
在数据插入时进行比较

In [3]:
class SortedPriorityQueue(PriorityQueueBase):
    #通过插入排序将最小元素放到队尾
    def InsertSort(self):
        for i in range(len(self.data)-2,-1,-1):
            if self.data[i]<self.data[i+1]:
                self.data[i],self.data[i+1]=self.data[i+1],self.data[i]
            else:
                break
                
    def __len__(self):
        return len(self.data)
    
    def __init__(self):
        self.data=[]
        
    def add(self,key,value):
        self.data.append(self._ltem(key,value))
        if len(self)>1:
            self.InsertSort()
    
    def min(self):
        if self.is_empty():
            raise Empty("no item")
        item=self.data[-1]
        return item._key,item._value
    
    def remove_min(self):
        if self.is_empty():
            raise Empty("no item")
        item=self.data.pop()
        return item._key,item._value
    
    def __iter__(self):
        for item in self.data:
            yield item._key,item._value
    
    def __str__(self):
        return " ".join([str(pair) for pair in iter(self)])
    
pq=SortedPriorityQueue()
pq.add(1,"a")
print(pq)
pq.add(3,"b")
print(pq)
print(pq.min()) 
pq.remove_min()
print(pq)

(1, 'a')
(3, 'b') (1, 'a')
(1, 'a')
(3, 'b')


比较两种优先队列的时间成本
![Screenshot from 2017-03-28 19-22-42.png](https://ooo.0o0.ooo/2017/03/28/58da4792edda8.png)

#### 二叉堆实现的优先队列
有序list的优先队列插入时间为$O(n)$  
无序list的优先队列删除成本为$O(n)$  
二叉堆实现的优先队列插入和删除的时间成本都是$O(logn)$

最小二叉堆特性:
1. 子节点的key大于或者小于其父节点
2. n节点的二叉树高度大于等于$logn$

插入新元素后恢复堆的顺序:**上浮法** 对新加入的节点,与它的父节点比较,小于父节点则交换
![Screenshot from 2017-03-28 20-11-08.png](https://ooo.0o0.ooo/2017/03/28/58da52f03a006.png)

删除根元素后恢复堆的顺序:**下沉法** ,将尾元素移到根,对根元素,除非没有子元素,比较它与子元素,大于则交换
![Screenshot from 2017-03-28 20-17-56.png](https://ooo.0o0.ooo/2017/03/28/58da54863f4cc.png)

用队列表示完全二叉树
![Screenshot from 2017-03-28 20-19-18.png](https://ooo.0o0.ooo/2017/03/28/58da54d6b3214.png)
我们从1开始编号,更方便

In [15]:
class HeapPriorityQuene(PriorityQueueBase):
    
    def parent(self,index):
        return index//2
    def left(self,index):
        return index*2
    def right(self,index):
        return index*2+1
    def swap(self,i,j):
        self.data[i],self.data[j]=self.data[j],self.data[i]
        
    def swim(self,j):
        while j>1 and self.data[j]<self.data[j//2]:
            self.swap(j,j//2)
            j=j//2
            
    def sink(self,j):
        while j<=len(self)//2:
            #只要左子树
            if j*2==len(self):
                nextj=j*2 
            elif self.data[j*2]<self.data[j*2+1]:
                nextj=j*2 
            else: 
                nextj=j*2+1
            if self.data[j]>self.data[nextj]:
                self.swap(j,nextj)
            else:break
            j=nextj
        
    def __init__(self):
        self.data=[0]
        
    def __len__(self):
        return len(self.data)-1
            
        
    def add(self,key,value):
        self.data.append(self._ltem(key,value))
        self.swim(len(self.data)-1)
        
    def min(self):
        if self.is_empty():
            raise Empty("no item")
        return self.data[1]._key,self.data[1]._value
    
    def remove_min(self):
        if self.is_empty():
            raise Empty("no item")
        minitem=self.data[1]
        self.swap(1,len(self))
        self.data.pop()
        self.sink(1)
        return minitem
    
    def __iter__(self):
        for item in self.data[1:]:
            yield item._key,item._value
    
    def __str__(self):
        return " ".join([str(pair) for pair in iter(self)])
    
pq=HeapPriorityQuene()
pq.add(1,"a")
print(pq)
pq.add(3,"b")
pq.add(0,"b")
print(pq)
print(pq.min()) 
pq.remove_min()
print(pq)

(1, 'a')
(0, 'b') (3, 'b') (1, 'a')
(0, 'b')
(1, 'a') (3, 'b')


基于heap的优先队列的分析:
空间:n  
时间:
![Screenshot from 2017-03-28 21-16-41.png](https://ooo.0o0.ooo/2017/03/28/58da624aadecd.png)

#### 堆建构 将无序list变成有序堆
for i in range(len(self)//2,0,-1):  
      sink(i)  
思路是从倒数第二层开始,对每个节点进行下沉

#### 堆排序  
修改sink,使之接受index 和 堆size
思路是先构造最大堆,再将堆顶与尾元素交换,再sink(1,size-1),直到size=0

In [24]:
class SortHeap:
    def __init__(self,l):
        self.data=[0]+l
        for i in range(len(self)//2,0,-1):
            self.sink(i,len(self))
            
    def __len__(self):
        return len(self.data)-1
    
    def swap(self,i,j):
        self.data[i],self.data[j]=self.data[j],self.data[i]
    
    def sink(self,j,size):
        while j<=size//2:
            if j*2==size:
                nextj=j*2 
            elif self.data[j*2]<self.data[j*2+1]:
                nextj=j*2 
            else: 
                nextj=j*2+1
            if self.data[j]>self.data[nextj]:
                self.swap(j,nextj)
            else:break
            j=nextj
            
    def sort(self):
        size=len(self)
        while size>1:
            self.swap(1,size)
            self.sink(1,size-1)
            size-=1
            
    def __iter(self):
        return lter(self.data)
    
    def __str__(self):
        return str(self.data)
            
            
s=SortHeap([1,6,3,4,5])
print(s)
s.sort()
print(s)

[0, 1, 4, 3, 6, 5]
[0, 6, 5, 4, 3, 1]


时间复杂度$O(nlogn)$